# MLOps Final Project:

Comparing performance of fine-tuned LLM vs out of the box pretrained LLM on sentiment analysis task for IMDB reviews dataset

# Data Preprocessing

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
import loralib as lora
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

df = pd.read_csv('movie_short.csv')

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_df['text'].tolist(), truncation=True, padding=True)

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_df['label'].tolist())
val_dataset = IMDbDataset(val_encodings, val_df['label'].tolist())

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred, average='binary')
    acc = accuracy_score(labels, pred)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


/opt/homebrew/anaconda3/envs/DistributedComputing/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Evaluation on pre-trained model

In [2]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=0,  # No training, just evaluation
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=10,
    do_eval=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

eval_result = trainer.evaluate()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/13 [00:00<?, ?it/s]

# Evaluation/training on fine-tuned model

In [3]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

eval_result_fine_tuned = trainer.evaluate()

  0%|          | 0/150 [00:00<?, ?it/s]

{'loss': 0.7043, 'grad_norm': 1.790898323059082, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}
{'loss': 0.6986, 'grad_norm': 1.6432048082351685, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.4}
{'loss': 0.6992, 'grad_norm': 0.9589131474494934, 'learning_rate': 3e-06, 'epoch': 0.6}
{'loss': 0.6944, 'grad_norm': 1.353496789932251, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.8}
{'loss': 0.6907, 'grad_norm': 0.9411355257034302, 'learning_rate': 5e-06, 'epoch': 1.0}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.6863863468170166, 'eval_accuracy': 0.655, 'eval_f1': 0.6960352422907489, 'eval_precision': 0.6124031007751938, 'eval_recall': 0.8061224489795918, 'eval_runtime': 4.4198, 'eval_samples_per_second': 45.251, 'eval_steps_per_second': 2.941, 'epoch': 1.0}
{'loss': 0.6867, 'grad_norm': 1.3369998931884766, 'learning_rate': 6e-06, 'epoch': 1.2}
{'loss': 0.6679, 'grad_norm': 2.5939736366271973, 'learning_rate': 7.000000000000001e-06, 'epoch': 1.4}
{'loss': 0.6652, 'grad_norm': 2.4567930698394775, 'learning_rate': 8.000000000000001e-06, 'epoch': 1.6}
{'loss': 0.634, 'grad_norm': 2.3099148273468018, 'learning_rate': 9e-06, 'epoch': 1.8}
{'loss': 0.5814, 'grad_norm': 2.3788418769836426, 'learning_rate': 1e-05, 'epoch': 2.0}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.5440200567245483, 'eval_accuracy': 0.835, 'eval_f1': 0.8390243902439024, 'eval_precision': 0.8037383177570093, 'eval_recall': 0.8775510204081632, 'eval_runtime': 4.3545, 'eval_samples_per_second': 45.93, 'eval_steps_per_second': 2.985, 'epoch': 2.0}
{'loss': 0.4935, 'grad_norm': 3.568794012069702, 'learning_rate': 1.1000000000000001e-05, 'epoch': 2.2}
{'loss': 0.3841, 'grad_norm': 2.9206383228302, 'learning_rate': 1.2e-05, 'epoch': 2.4}
{'loss': 0.409, 'grad_norm': 8.060220718383789, 'learning_rate': 1.3000000000000001e-05, 'epoch': 2.6}
{'loss': 0.3572, 'grad_norm': 16.010934829711914, 'learning_rate': 1.4000000000000001e-05, 'epoch': 2.8}
{'loss': 0.3121, 'grad_norm': 6.208229064941406, 'learning_rate': 1.5e-05, 'epoch': 3.0}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.3339410126209259, 'eval_accuracy': 0.875, 'eval_f1': 0.8633879781420765, 'eval_precision': 0.9294117647058824, 'eval_recall': 0.8061224489795918, 'eval_runtime': 4.3669, 'eval_samples_per_second': 45.799, 'eval_steps_per_second': 2.977, 'epoch': 3.0}
{'train_runtime': 184.0001, 'train_samples_per_second': 13.027, 'train_steps_per_second': 0.815, 'train_loss': 0.5785508537292481, 'epoch': 3.0}


  0%|          | 0/13 [00:00<?, ?it/s]

# Evaluation/training on LoRA fine-tuned model

In [4]:
lora_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules= ["attention.q_lin", "attention.k_lin", "attention.v_lin"], #["query", "key", "value"],  # Specify the target modules
    bias="none",
)

lora_model = get_peft_model(lora_model, lora_config)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

eval_result_lora = trainer.evaluate()

/opt/homebrew/anaconda3/envs/DistributedComputing/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/150 [00:00<?, ?it/s]

{'loss': 0.2394, 'grad_norm': 1.6503535509109497, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}
{'loss': 0.2292, 'grad_norm': 10.250137329101562, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.4}
{'loss': 0.187, 'grad_norm': 7.581302165985107, 'learning_rate': 3e-06, 'epoch': 0.6}
{'loss': 0.1737, 'grad_norm': 2.624061346054077, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.8}
{'loss': 0.1746, 'grad_norm': 6.689779758453369, 'learning_rate': 5e-06, 'epoch': 1.0}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.3054206967353821, 'eval_accuracy': 0.88, 'eval_f1': 0.8736842105263158, 'eval_precision': 0.9021739130434783, 'eval_recall': 0.8469387755102041, 'eval_runtime': 4.3753, 'eval_samples_per_second': 45.711, 'eval_steps_per_second': 2.971, 'epoch': 1.0}
{'loss': 0.1784, 'grad_norm': 9.469088554382324, 'learning_rate': 6e-06, 'epoch': 1.2}
{'loss': 0.1752, 'grad_norm': 12.691534042358398, 'learning_rate': 7.000000000000001e-06, 'epoch': 1.4}
{'loss': 0.1808, 'grad_norm': 10.022178649902344, 'learning_rate': 8.000000000000001e-06, 'epoch': 1.6}
{'loss': 0.0938, 'grad_norm': 3.054330825805664, 'learning_rate': 9e-06, 'epoch': 1.8}
{'loss': 0.1237, 'grad_norm': 6.956212997436523, 'learning_rate': 1e-05, 'epoch': 2.0}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.3119381070137024, 'eval_accuracy': 0.89, 'eval_f1': 0.8865979381443299, 'eval_precision': 0.8958333333333334, 'eval_recall': 0.8775510204081632, 'eval_runtime': 4.3477, 'eval_samples_per_second': 46.002, 'eval_steps_per_second': 2.99, 'epoch': 2.0}
{'loss': 0.0989, 'grad_norm': 5.5703582763671875, 'learning_rate': 1.1000000000000001e-05, 'epoch': 2.2}
{'loss': 0.0837, 'grad_norm': 0.9308735728263855, 'learning_rate': 1.2e-05, 'epoch': 2.4}
{'loss': 0.093, 'grad_norm': 0.22846873104572296, 'learning_rate': 1.3000000000000001e-05, 'epoch': 2.6}
{'loss': 0.0778, 'grad_norm': 1.810604214668274, 'learning_rate': 1.4000000000000001e-05, 'epoch': 2.8}
{'loss': 0.0692, 'grad_norm': 1.0725218057632446, 'learning_rate': 1.5e-05, 'epoch': 3.0}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.3993997275829315, 'eval_accuracy': 0.9, 'eval_f1': 0.8979591836734694, 'eval_precision': 0.8979591836734694, 'eval_recall': 0.8979591836734694, 'eval_runtime': 4.3533, 'eval_samples_per_second': 45.942, 'eval_steps_per_second': 2.986, 'epoch': 3.0}
{'train_runtime': 181.015, 'train_samples_per_second': 13.242, 'train_steps_per_second': 0.829, 'train_loss': 0.14522642691930135, 'epoch': 3.0}


  0%|          | 0/13 [00:00<?, ?it/s]

# Results

In [5]:
print("Pre-trained DistilBERT evaluation:")
print(eval_result)

print("Regular fine-tuned DistilBERT evaluation:")
print(eval_result_fine_tuned)

print("LoRA fine-tuned DistilBERT evaluation:")
print(eval_result_lora)

Pre-trained DistilBERT evaluation:
{'eval_loss': 0.6990486979484558, 'eval_accuracy': 0.49, 'eval_f1': 0.6577181208053692, 'eval_precision': 0.49, 'eval_recall': 1.0, 'eval_runtime': 4.6223, 'eval_samples_per_second': 43.268, 'eval_steps_per_second': 2.812}
Regular fine-tuned DistilBERT evaluation:
{'eval_loss': 0.3339410126209259, 'eval_accuracy': 0.875, 'eval_f1': 0.8633879781420765, 'eval_precision': 0.9294117647058824, 'eval_recall': 0.8061224489795918, 'eval_runtime': 4.3261, 'eval_samples_per_second': 46.231, 'eval_steps_per_second': 3.005, 'epoch': 3.0}
LoRA fine-tuned DistilBERT evaluation:
{'eval_loss': 0.3993997275829315, 'eval_accuracy': 0.9, 'eval_f1': 0.8979591836734694, 'eval_precision': 0.8979591836734694, 'eval_recall': 0.8979591836734694, 'eval_runtime': 4.3293, 'eval_samples_per_second': 46.197, 'eval_steps_per_second': 3.003, 'epoch': 3.0}
